On tente d'extraire les fichiers JSON ici

In [2]:
import json
import pandas as pd
import numpy as np

Pour rappel je me base à fond sur le cours de INFO-H600 pour l'extraction du JSON vers un panda DataFrame

Il est temps de transférer tout les fichiers JSON en DataFrame

In [3]:
#Run time is around 9 min for me
files_name = ['vehiclePosition01.json','vehiclePosition02.json','vehiclePosition03.json','vehiclePosition04.json','vehiclePosition05.json','vehiclePosition06.json','vehiclePosition07.json','vehiclePosition08.json','vehiclePosition09.json','vehiclePosition10.json','vehiclePosition11.json','vehiclePosition12.json','vehiclePosition13.json']
ListOfData=[]
for l,name in enumerate(files_name):
    with open(name) as f:
        d=json.load(f)
        ListOfData.append(d)
        f.close()
    print('Fichier '+str(l+1)+ ': Done')

Fichier 1: Done
Fichier 2: Done
Fichier 3: Done
Fichier 4: Done
Fichier 5: Done
Fichier 6: Done
Fichier 7: Done
Fichier 8: Done
Fichier 9: Done
Fichier 10: Done
Fichier 11: Done
Fichier 12: Done
Fichier 13: Done


In [4]:
#We start cleaning, run time is 25 sec

for k, data in enumerate(ListOfData):
    # Here I clean every ApiCall (i.e. for each time) that contains nothing or is assigned to nothing (None)
    data_cleaned = [ApiCall for ApiCall in data['data'] 
    if ((ApiCall is not None) & (ApiCall['Responses'] is not None ) &(ApiCall['Responses'] is not [] ) )
    ]

    # Then we go to clean the 'Responses' to remove the None value

    for i,ApiCall in enumerate(data_cleaned):
        data_cleaned[i]['Responses'] = [response for response in ApiCall['Responses'] if (response is not None) ]

    # Once it's done, we go to the 'lines' key and we again remove the None value

    for i,ApiCall in enumerate(data_cleaned):
        data_cleaned[i]['Responses'] = [response for response in ApiCall['Responses'] 
        if (response['lines'] is not None) 
        ]
    ListOfData[k]=data_cleaned
    
    print('Fichier '+str(k+1)+ ': Done') 
    # Data should be clean by now

Fichier 1: Done
Fichier 2: Done
Fichier 3: Done
Fichier 4: Done
Fichier 5: Done
Fichier 6: Done
Fichier 7: Done
Fichier 8: Done
Fichier 9: Done
Fichier 10: Done
Fichier 11: Done
Fichier 12: Done
Fichier 13: Done


Ok maintenant on passe au gros morceau

In [5]:
#Runtime ~ 3min

ListOfDataFrame=[]

for k,data_cleaned in enumerate(ListOfData):

    vehiclePosition= pd.json_normalize(data=data_cleaned,record_path=['Responses','lines','vehiclePositions'],
        meta=['time',['Responses','lines','lineId']],errors='ignore')
    vehiclePosition.columns = ['directionId', 'distanceFromPoint','pointId','time','lineId']
    ListOfDataFrame.append(vehiclePosition)
    print('Fichier '+str(k+1)+ ': Done') 
#Change the name of the column so that it's easy

Fichier 1: Done
Fichier 2: Done
Fichier 3: Done
Fichier 4: Done
Fichier 5: Done
Fichier 6: Done
Fichier 7: Done
Fichier 8: Done
Fichier 9: Done
Fichier 10: Done
Fichier 11: Done
Fichier 12: Done
Fichier 13: Done


On peut mtn les concaténer tout ensemble

In [6]:
#Runtime ~ 4 min

GlobalVehiclePosition = pd.concat(ListOfDataFrame, axis=0)

In [55]:
GlobalVehiclePosition['time']=GlobalVehiclePosition['time'].astype('int64')

In [8]:
len(GlobalVehiclePosition)

19421883

In [56]:
GlobalVehiclePosition.time

0         1630914886924
1         1630914886924
2         1630914886924
3         1630914886924
4         1630914886924
              ...      
284009    1632241153806
284010    1632241153806
284011    1632241153806
284012    1632241153806
284013    1632241153806
Name: time, Length: 19421883, dtype: int64

Construction Dictionnaire JSON 

In [9]:
import gtfs_functions as gtfs
import matplotlib
import geopandas

routes, stops, stop_times, trips, shapes = gtfs.import_gtfs("gtfs3Sept.zip")
stop_freq = gtfs.stops_freq(stop_times, stops)
line_freq = gtfs.lines_freq(stop_times, trips, shapes, routes)


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import random

In [20]:
ListOfStop = stops['stop_id'].unique()
len(ListOfStop)

2376

In [10]:
def FromRouteIdtoLineId(RouteId):
    routeId = routes[routes['route_id']==RouteId]['route_short_name'].iloc[0]
    return routeId

In [35]:
dico = []
i=0
j=0
for stop in ListOfStop:
    i+=1
    RoutePassingByStop = stop_times[stop_times['stop_id']==stop]['route_id'].unique()
    LinePassingByStop = [FromRouteIdtoLineId(route) for route in RoutePassingByStop ]
    dico.append({'stop_id':stop,'lines':LinePassingByStop})
    if i==100:
        j+=1
        print('Nombre arret fait:' +str(j*100))
        i=0


Nombre arret fait:100
Nombre arret fait:200
Nombre arret fait:300
Nombre arret fait:400
Nombre arret fait:500
Nombre arret fait:600
Nombre arret fait:700
Nombre arret fait:800
Nombre arret fait:900
Nombre arret fait:1000
Nombre arret fait:1100
Nombre arret fait:1200
Nombre arret fait:1300
Nombre arret fait:1400
Nombre arret fait:1500
Nombre arret fait:1600
Nombre arret fait:1700
Nombre arret fait:1800
Nombre arret fait:1900
Nombre arret fait:2000
Nombre arret fait:2100
Nombre arret fait:2200
Nombre arret fait:2300


Code Medhi

In [48]:
df_vehiculepositions = GlobalVehiclePosition

In [58]:
df_vehiculepositions['time'] = pd.to_numeric(df_vehiculepositions['time'], downcast = "float")

##Division par 1000 pour passer de ms à s /!\ Attention à le faire qu'une fois
df_vehiculepositions['time'] = df_vehiculepositions['time']/1000 

In [59]:
df_dict = df_vehiculepositions.to_dict()

In [65]:
df_dict['time']

{0: 1632229376.0,
 1: 1632229376.0,
 2: 1632229376.0,
 3: 1632229376.0,
 4: 1632229376.0,
 5: 1632229376.0,
 6: 1632229376.0,
 7: 1632229376.0,
 8: 1632229376.0,
 9: 1632229376.0,
 10: 1632229376.0,
 11: 1632229376.0,
 12: 1632229376.0,
 13: 1632229376.0,
 14: 1632229376.0,
 15: 1632229376.0,
 16: 1632229376.0,
 17: 1632229376.0,
 18: 1632229376.0,
 19: 1632229376.0,
 20: 1632229376.0,
 21: 1632229376.0,
 22: 1632229376.0,
 23: 1632229376.0,
 24: 1632229376.0,
 25: 1632229376.0,
 26: 1632229376.0,
 27: 1632229376.0,
 28: 1632229376.0,
 29: 1632229376.0,
 30: 1632229376.0,
 31: 1632229376.0,
 32: 1632229376.0,
 33: 1632229376.0,
 34: 1632229376.0,
 35: 1632229376.0,
 36: 1632229376.0,
 37: 1632229376.0,
 38: 1632229376.0,
 39: 1632229376.0,
 40: 1632229376.0,
 41: 1632229376.0,
 42: 1632229376.0,
 43: 1632229376.0,
 44: 1632229376.0,
 45: 1632229376.0,
 46: 1632229376.0,
 47: 1632229376.0,
 48: 1632229376.0,
 49: 1632229376.0,
 50: 1632229376.0,
 51: 1632229376.0,
 52: 1632229376.0,
 53

In [61]:
import datetime
from tqdm import tqdm

def datefromtimestamp(timestamp):
    date = datetime.datetime.fromtimestamp(timestamp).strftime('%m-%d-%Y %H:%M:%S')
    return date

def actual_time(line, stop, year, month, day):
    
    date_init = datetime.datetime(year, month, day, 4)
    epoch_date_init = datetime.datetime.timestamp(date_init)
    date_end = datetime.datetime(year, month, day+1, 3) #L'idée c'est de commencer à 4h du mat et finir à 3h du matin le lendemain pour inclure les transports après minuit
    epoch_date_end = datetime.datetime.timestamp(date_end)
    date_midnight = datetime.datetime(year, month, day, 0) #Obtain seconds after midnight format
    midnight_time = datetime.datetime.timestamp(date_midnight)
    listvehiculearrivals = []
    humanlistvehiculearrivals = []
    for index in tqdm(df_dict['time']):
        if epoch_date_init < df_dict['time'][index]:
            if epoch_date_end > df_dict['time'][index]:
                if df_dict['lineId'][index] == line:
                    if df_dict['pointId'][index] == stop:
                        if df_dict['distanceFromPoint'][index] == 0:
                            sec_after_midnight = df_dict['time'][index] -midnight_time
                            human_arrival_time = datefromtimestamp(df_dict['time'][index])
                            listvehiculearrivals.append(sec_after_midnight)
                            humanlistvehiculearrivals.append(human_arrival_time)
    return listvehiculearrivals, humanlistvehiculearrivals

timetable, machine_timetable = actual_time("4", "5026", 2021, 9, 9)
timetable

100%|██████████| 3548922/3548922 [00:01<00:00, 2192818.51it/s]


[]

In [39]:
import gtfs_functions as gtfs
import matplotlib
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import random

routes, stops, stop_times, trips, shapes = gtfs.import_gtfs("gtfs3Sept.zip")
stop_freq = gtfs.stops_freq(stop_times, stops)
line_freq = gtfs.lines_freq(stop_times, trips, shapes, routes)

In [40]:
import datetime

calendardf = pd.read_table('calendar.txt',sep=',')          #export the calendar file into data frame

def CheckRange(Start,End,date):
    return int(date) in range(int(Start),int(End))

def FromDayToService(date):
    #date est un tuple ('20210907','tuesday')
    ListServiceId=[]

    for i in range(len(calendardf)):

        startdate,endate=calendardf.iloc[i,[8,9]]
        
        if CheckRange(startdate,endate+1,date[0]):

            if calendardf.loc[i,date[1]]==1:
                
                ListServiceId.append(str(calendardf.iloc[i,0]))

    return ListServiceId

def FromLineIdtoRouteId(LineId):
    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId

def SelectInGTFS(day,lineId,stopid):
    ListOfService = FromDayToService(day)
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopid)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    Comparaisondf=Comparaisondf[['trip_id','arrival_time','stop_id','route_id','service_id']]
    ArrivalTime = Comparaisondf['arrival_time'].values
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime.insert(0,ArrivalTime[0])
    Comparaisondf['Interval']= IntervalTime
    Comparaisondf['Ponctuality']=np.array(IntervalTime)>720
    return Comparaisondf

def ScheduledTime(lineId,stopId,date):
    ListOfService = FromDayToService(date)
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Comparaisondf['arrival_time'].values
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime.insert(0,ArrivalTime[0])
    Punctuality = np.array(IntervalTime)>720 
    IntervalTime = list(zip(IntervalTime,Punctuality))

    return IntervalTime

In [45]:
def actual_timePierre(dataframe,lineId, stopId, date):
    year = int(date[0][:4])
    month = int(date[0][4:6])
    day = int(date[0][6:8]) 
    ListOfService = FromDayToService(date)
    RouteId = FromLineIdtoRouteId(lineId)
    date_init = datetime.datetime(year, month, day, 4)
    epoch_date_init = int(datetime.datetime.timestamp(date_init))
    date_end = datetime.datetime(year, month, day+1, 3) #L'idée c'est de commencer à 4h du mat et finir à 3h du matin le lendemain pour inclure les transports après minuit
    epoch_date_end = int(datetime.datetime.timestamp(date_end))
    dataframe['time']=dataframe['time'].astype('int64')
    df = dataframe[(dataframe['time'] >= epoch_date_init)&
                            (dataframe['time'] <= epoch_date_end)&
                                (dataframe['lineId']==lineId)
                            &(dataframe['pointId']==stopId)
                            &(dataframe['distanceFromPoint']==0)]

    return df
    #,epoch_date_end

In [41]:
Date=('20210910','friday')
lineid = '4'
stopid = '5026'

In [46]:
actual_timePierre(ListOfDataFrame[9],lineid,stopid,Date)

,directionId,distanceFromPoint,pointId,time,lineId


In [47]:
for i,dataframe in enumerate(ListOfDataFrame):
    variable = actual_timePierre(dataframe,lineid,stopid,Date)
    print('Fichier '+str(i)+': done\n Longueur: '+str(len(variable)))

Fichier 0: done
 Longueur: 0
Fichier 1: done
 Longueur: 0
Fichier 2: done
 Longueur: 0
Fichier 3: done
 Longueur: 0
Fichier 4: done
 Longueur: 0
Fichier 5: done
 Longueur: 0
Fichier 6: done
 Longueur: 0
Fichier 7: done
 Longueur: 0
Fichier 8: done
 Longueur: 0
Fichier 9: done
 Longueur: 0
Fichier 10: done
 Longueur: 0
Fichier 11: done
 Longueur: 0
Fichier 12: done
 Longueur: 0


In [33]:
for k in range(len(ListOfDataFrame)):
    print(ListOfDataFrame[k][(ListOfDataFrame[k]['time'].astype('int64')<1631062800)])

Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []
Empty DataFrame
Columns: [directionId, distanceFromPoint, pointId, time, lineId]
Index: []